In [ ]:
%matplotlib inline

#puts above directory into the path
import sys
sys.path.append("../../VREP_robot")
sys.path.append("../../VREP_robot/utils")
sys.path.append("../")
import vrep
from vrepRobot import *

from forwardKinematics import robot_config
import numpy as np
import time

pi = np.pi

In [2]:
#puts above directory into the path
import sys
sys.path.append("../../VREP_robot")
sys.path.append("../../VREP_robot/utils")
sys.path.append("../")
import vrep
from vrepRobot import *

from forwardKinematics import robot_config
import numpy as np
import time

pi = np.pi

In [ ]:
# setup environment
myRobot = robot_config()
vrep_env = VREP_Environement(synchronous=False)
ik_handles = ['ik_joint1', 'ik_joint2', 'ik_joint3', 'ik_joint4', 'ik_joint5', 'ik_joint6', 'ik_joint7', 'kinematicsTest_IKTip', 'ik_rf7_static']
#k_handles  = ['pose_j0', 'pose_j1', 'pose_j2', 'pose_j3', 'pose_j4', 'pose_j5', 'pose_j6', 'pose_j7']
vrep_env.add_robot(VREP_Robot('ik_robot', ik_handles, connection_type = 'blocking'))
vrep_env.start_simulation()

Connected to remote API server
1 robot(s) connected: ['ik_robot']


In [ ]:
# generate config samples
from configGenerator_VREP import configGenerator_VREP
config = configGenerator_VREP.generate_sample(2)  # set sample number here
total_config = config.shape[0]
print("total configs = ", total_config)
#print(config)
print(total_config/50/3600)

In [ ]:
# initialize collision state placeholder
y = np.zeros((total_config))
EEPosition = np.zeros((total_config,3))
#EEorientation = np.zeros((total_config,3))
EEQuaternion = np.zeros((total_config,4))

In [ ]:
# get collision handle
vrep_env.ik_robot.getCollisionHandle('Collision')
vrep.simxSynchronousTrigger(vrep_env.clientID)
vrep.simxGetPingTime(vrep_env.clientID)

time.sleep(10)

In [ ]:
# initialize collision check
vrep_env.ik_robot.getCollisionState(initialize=True)

In [ ]:
# collision detect
from tqdm import tqdm_notebook as tqdm

pbar = tqdm(total=total_config)  # set progress bar

t = time.clock()
for i in range(total_config):
#for i in range(len(jointAngle)):
    joint_angle = config[i, :]
    for j in range(6):
        vrep_env.ik_robot.setJointPosition(vrep_env.ik_robot.handles[j], joint_angle[j])
    #vrep_env.ik_robot.setJointPosition(vrep_env.ik_robot.handles[5], jointAngle[i])
    
    vrep.simxSynchronousTrigger(vrep_env.clientID)
    vrep.simxGetPingTime(vrep_env.clientID)
    collisionState = vrep_env.ik_robot.getCollisionState()
    y[i] = collisionState
    #vrep.simxSynchronousTrigger(vrep_env.clientID)
    #vrep.simxGetPingTime(vrep_env.clientID)
    EEPosition[i,:] = vrep_env.ik_robot.getObjectPosition(vrep_env.ik_robot.handles[8], relative2='ik_rf7_static')[1]
    EEQuaternion[i,:] = vrep_env.ik_robot.getObjectQuaternion(vrep_env.ik_robot.handles[8], relative2='ik_rf7_static')[1]
    pbar.update(1)
    #time.sleep(0.5)
    #print(EEPosition[i,:])
    
    if i%int(total_config/100+1) == 0:
        Data = np.hstack((config, y.reshape(-1,1),EEPosition,EEQuaternion))
        #np.save('logs/CollisionData{}.npy'.format(i),Data)
    
pbar.close()
print('runtime = ', time.clock()-t)  
# print('collision state = ', y)
print('EEposition\n', EEPosition)
# print('EEorientation\n', EEorientation)

In [ ]:
# save data
Data = np.hstack((config, y.reshape(-1,1),EEPosition,EEQuaternion))
print(Data)
np.save('CollisionData.npy',Data)

In [ ]:
loadedData = np.load('CollisionData.npy')
print(loadedData)

In [ ]:
# plot

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(EEPosition[:,0], EEPosition[:,1], EEPosition[:,2])
plt.show()

In [ ]:
# test
# for i in range(100):
#     joint_angle = np.sin(i*2*np.pi/100)*np.pi/4
#     vrep_env.ik_robot.setJointPosition(vrep_env.ik_robot.handles[5], joint_angle)
#     vrep.simxSynchronousTrigger(vrep_env.clientID)
#     vrep.simxGetPingTime(vrep_env.clientID)
#     collisionState = vrep_env.ik_robot.getCollisionState()
#     print(collisionState)

In [ ]:
np.unique(y)